In [1]:
import pandas as pd
from pandas.tseries.offsets import Week

In [2]:
hist_tx_df = pd.read_csv("evictions/tx_weekly_hist_dta_all.csv")
recent_hou_df = pd.read_csv("evictions/houston_df_clean3.csv")
recent_hou_df_2 = pd.read_csv("evictions/df_houston_clean3_jan18tojul18.csv")
recent_dal_df = pd.read_csv("evictions/cleaned_dallas_records.csv")

/Users/joshnoble/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (19,42,50,51,52,53,55,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/joshnoble/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (19,54,62,63,65,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import datetime

recent_weeks_generated = pd.DataFrame()
recent_weeks_generated['week_dt'] = pd.date_range(start = '2018-01-01', end = '2021-09-13', freq='W-SUN')
recent_weeks_generated['week'] = recent_weeks_generated['week_dt'].dt.isocalendar().week
recent_weeks_generated['year'] = recent_weeks_generated['week_dt'].dt.year


all_weeks_generated = pd.DataFrame()
all_weeks_generated['week_dt'] = pd.date_range(start = '2001-01-01', end = '2021-09-13', freq='W')
all_weeks_generated['week'] = all_weeks_generated['week_dt'].dt.isocalendar().week
all_weeks_generated['year'] = all_weeks_generated['week_dt'].dt.year
#all_weeks_generated['weeksfromstart'] = ((all_weeks_generated['week_dt'].dt.year - 2001) * 52) + all_weeks_generated['week'].astype(int)

In [1417]:
all_weeks_generated[all_weeks_generated.duplicated(subset=['week', 'year'], keep=False)]

,week_dt,week,year
260,2006-01-01,52,2006
312,2006-12-31,52,2006
573,2012-01-01,52,2012
625,2012-12-30,52,2012
834,2017-01-01,52,2017
886,2017-12-31,52,2017


In [4]:
all_weeks_generated.loc[(all_weeks_generated['week_dt'].dt.month == 1)
                        & (all_weeks_generated['week'] == 52), 'week'] = 0

In [5]:
all_weeks_generated.loc[(all_weeks_generated['week_dt'].dt.month == 12)
                        & (all_weeks_generated['week'] == 1), 'week'] = 52

In [6]:
all_weeks_generated[all_weeks_generated.duplicated(subset=['week', 'year'], keep=False)]

,week_dt,week,year


In [1421]:
dtest = pd.to_datetime('2019-01-01')
dtest.week

1

In [7]:
recent_dal_df['datetime'] = pd.to_datetime(recent_dal_df['date'])
recent_dal_df['week'] = recent_dal_df['datetime'].dt.isocalendar().week
recent_dal_df.loc[(recent_dal_df['datetime'].dt.month == 1) & (recent_dal_df['week'] == 52), 'week'] = 0
recent_dal_df['year'] = pd.to_datetime(recent_dal_df['date']).dt.year

# for row in recent_dal_df.itertuples():
#     for dt in recent_weeks_generated:
#         #new_houston.at[row.Index,'week']
#         w = recent_dal_df.at[row.Index,'week']
#         y = recent_dal_df.at[row.Index, 'year']
#         #week_match = (dt.week == w)
#         if( dt.week is w and dt.year is y):
#             recent_dal_df.at[row.Index, 'new_week_dt'] = dt


recent_dal_df = recent_dal_df.merge(all_weeks_generated, on=['week', 'year'])

In [8]:
recent_dal_df.head()

,Unnamed: 0,case_number,date,amount,precinct_id,council_id,tract_id,zip_id,city_id,county_id,lon,lat,datetime,week,year,week_dt
0,0,1804351H,2018-07-10,1590.0,48113-1-1,4819000-07,48113003400,75215.0,4819000.0,48113,-96.780003,32.762274,2018-07-10,28,2018,2018-07-15
1,1,1804378H,2018-07-10,1217.0,48113-1-1,4819000-07,48113003700,75215.0,4819000.0,48113,-96.757373,32.767008,2018-07-10,28,2018,2018-07-15
2,3,1804376H,2018-07-10,700.0,48113-1-1,4819000-07,48113002701,75210.0,4819000.0,48113,-96.743307,32.775385,2018-07-10,28,2018,2018-07-15
3,4,1804375H,2018-07-10,700.0,48113-1-1,4819000-07,48113002701,75210.0,4819000.0,48113,-96.742441,32.776028,2018-07-10,28,2018,2018-07-15
4,5,1804377H,2018-07-10,656.0,48113-1-1,4819000-14,48113003101,75201.0,4819000.0,48113,-96.798797,32.781548,2018-07-10,28,2018,2018-07-15


In [9]:
recent_dal_df = recent_dal_df[(recent_dal_df['tract_id'] > 48113000000) & 
                              (recent_dal_df['tract_id'] < 48114000000)]

In [10]:

weeks_18_21 = all_weeks_generated[all_weeks_generated.year > 2017]

recent_dal_ts = recent_dal_df.groupby(['week_dt', 'tract_id']).size().sort_values(ascending=False).reset_index(name='count') 

all_weeks = recent_dal_df.week_dt.unique()
all_tracts = []

# we did want to do all weeks Dallas has, but there's weird gaps
for week in weeks_18_21['week_dt'].unique():
    all_tracts.extend(recent_dal_df.tract_id.unique())

all_tracts.sort()

all_weeks = []
for tract in recent_dal_df.tract_id.unique():
    all_weeks.extend(weeks_18_21['week_dt'].unique())
    #all_weeks.extend(recent_dal_df.week_dt.unique())

all_dallas = pd.DataFrame({'week_dt':all_weeks, 'tract_id': all_tracts})
all_dallas.head()

,week_dt,tract_id
0,2018-01-07,48113000100
1,2018-01-14,48113000100
2,2018-01-21,48113000100
3,2018-01-28,48113000100
4,2018-02-04,48113000100


In [11]:
all_dallas = all_dallas.merge(recent_dal_ts, how='left', on=['week_dt', 'tract_id'])

In [12]:
all_dallas['count'] = all_dallas['count'].fillna(0)
all_dallas = all_dallas.rename(columns={'tract_id':"GEOID"})

In [13]:
all_dallas = all_dallas[all_dallas['week_dt'] > '12-31-2017']

In [14]:
print(len(all_dallas['week_dt'].unique()))

193


In [15]:
all_dallas['count'].max()

75.0

# Houston has no Census tracts, so we need to break them into chunks of 10,000 records, upload to https://geocoding.geo.census.gov/geocoder/locations/addressbatch?form and then get the results back.

In [16]:
geocodes = ["geocodes/GeocodeResults13.csv",
            "geocodes/GeocodeResults14.csv",
            "geocodes/GeocodeResults15.csv",
            "geocodes/GeocodeResults12.csv",
            "geocodes/GeocodeResults.csv",
            "geocodes/GeocodeResults1.csv",
            "geocodes/GeocodeResults2.csv",
            "geocodes/GeocodeResults3.csv",
            "geocodes/GeocodeResults4.csv",
            "geocodes/GeocodeResults5.csv",
            "geocodes/GeocodeResults6.csv",
            "geocodes/GeocodeResults7.csv",
            "geocodes/GeocodeResults8.csv",
            "geocodes/GeocodeResults9.csv",
            "geocodes/GeocodeResults10.csv",
            "geocodes/GeocodeResults11.csv"]

all_geocodes = []

import io

addr_to_ct = ["Defendant Addr Line 1,GEOID"]

for g in geocodes:
    lines=open(g,"r").read().splitlines()
    ok_lines = []
    for i in range(len(lines)):
        if ('No_Match' not in lines[i]) and ('Tie' not in lines[i]):
            ok_lines.append(lines[i])

    for l in ok_lines:
        vals = l.split(',')
        address = vals[1].replace("\"", "")
        tract = str(vals[-4]).replace("\"", "") + str(vals[-3]).replace("\"", "") + str(vals[-2]).replace("\"", "")
        addr_to_ct.append("\"" + address + "\"," + tract)

In [17]:
add_ct_df = pd.read_csv(io.StringIO('\n'.join(addr_to_ct)))

# Now we have all the rows with their census tracts, we can load them all back in and do our merge with the original datasets.

In [18]:
#hou_address_to_ct = pd.read_csv("hou_add_ct.csv")
hou_address_to_ct = add_ct_df
hou_address_to_ct.head()

,Defendant Addr Line 1,GEOID
0,8001 W. Tidwell Rd,48201521702
1,14501 Empanada Dr #817,48201452703
2,8001 W Tidwell Rd,48201521702
3,10580 Hammerly Blvd Apt #1806,48201521900
4,3833 Cummins St,48201411506


In [19]:
import numpy as np

recent_hou_df = recent_hou_df.rename(columns={'Defendant Addr Line 1 ': 'Defendant Addr Line 1'})
recent_hou_df = recent_hou_df.merge(hou_address_to_ct, on='Defendant Addr Line 1')

In [1434]:
recent_hou_df_2 = recent_hou_df_2.rename(columns={'Defendant Addr Line 1 ': 'Defendant Addr Line 1'})
recent_hou_df_2 = recent_hou_df_2.merge(hou_address_to_ct, on='Defendant Addr Line 1')

In [20]:
new_houston = pd.concat([recent_hou_df, recent_hou_df_2])
new_houston = new_houston.dropna(subset=['GEOID'], how='all')
new_houston = new_houston.astype({"GEOID": int})

In [21]:
# make a temp column with the dates

new_houston['datetime'] = pd.to_datetime(new_houston['Case File Date'])
new_houston['week'] = pd.to_datetime(new_houston['Case File Date']).dt.isocalendar().week
new_houston['year'] = pd.to_datetime(new_houston['Case File Date']).dt.year
new_houston.loc[(new_houston['datetime'].dt.month == 1) & (new_houston['week'] == 52), 'week'] = 0


In [22]:
new_houston = new_houston.drop_duplicates(subset=['Case Number'])
new_houston = new_houston.drop('Unnamed: 0', 1)
new_houston.head()

/var/folders/nv/rrtmvdnd3q7bt4yc121xcvx00000gn/T/ipykernel_79523/2816990293.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  new_houston = new_houston.drop('Unnamed: 0', 1)


,Attorney Fees,Case File Date,Case Number,Case Status,Case Subtype,Case Type,Claim Amount,Court Costs,Defendant Addr City,Defendant Addr Line 1,...,Second Plaintiff Atty State,Second Plaintiff Atty ZIP 1,Second Plaintiff Name,Style Of Case,file,GEOID,Defendant Addr Line 1,datetime,week,year
0,0.0,2019-01-02,191200000920,Disposed,Eviction,CV,1293.81,121.0,Houston,4411 Fulton St. # 60,...,NaN,NaN,NaN,SIGI ASSET 1 LLC THE STATION vs. JOSE LUIS CON...,CasesEntered-01012019To01312019(1).txt,48201210600,NaN,2019-01-02,1,2019
1,100.0,2019-01-02,191200003322,Disposed,Eviction,CV,363.00,121.0,Houston,1517 Andrews Street #A,...,NaN,NaN,NaN,HISTORIC RENTAL INITIATIVE vs. De 'Shay Ross,CasesEntered-01012019To01312019(1).txt,48201410101,NaN,2019-01-02,1,2019
2,0.0,2019-01-02,191200003368,Disposed,Eviction,CV,303.00,121.0,Houston,805 Valentine Way #P,...,NaN,NaN,NaN,Historic Oaks of Allen Parkway Apartments vs. ...,CasesEntered-01012019To01312019(1).txt,48201410102,NaN,2019-01-02,1,2019
3,0.0,2019-01-02,191200003373,Disposed,Eviction,CV,556.00,196.0,Houston,1315 Park Trail Run,...,NaN,NaN,NaN,Historic Oaks of Allen Parkway Apartments vs. ...,CasesEntered-01012019To01312019(1).txt,48201410102,NaN,2019-01-02,1,2019
4,100.0,2019-01-02,191200003377,Disposed,Eviction,CV,244.00,121.0,Houston,814 Skyline Trail,...,NaN,NaN,NaN,Historic Oaks of Allen Parkway Apartments vs. ...,CasesEntered-01012019To01312019(1).txt,48201410102,NaN,2019-01-02,1,2019


In [23]:
new_houston = new_houston.merge(all_weeks_generated, on=['week', 'year'])

In [24]:
weeks_18_21 = all_weeks_generated[all_weeks_generated.year > 2017]
recent_hou_ts = new_houston.groupby(['week_dt', 'GEOID']).size().sort_values(ascending=False).reset_index(name='count') 

all_weeks = recent_hou_ts.week_dt.unique()
all_tracts = []

# we were using the dates in Houstons values, but there were weird gaps
for week in weeks_18_21['week_dt'].unique():
    all_tracts.extend(recent_hou_ts.GEOID.unique())

all_tracts.sort()

all_weeks = []
for tract in recent_hou_ts.GEOID.unique():
    all_weeks.extend(weeks_18_21['week_dt'].unique())

all_houston = pd.DataFrame({'week_dt':all_weeks, 'GEOID': all_tracts})

In [25]:
all_houston.head()

,week_dt,GEOID
0,2018-01-07,48157670101
1,2018-01-14,48157670101
2,2018-01-21,48157670101
3,2018-01-28,48157670101
4,2018-02-04,48157670101


In [26]:
all_houston = all_houston.merge(recent_hou_ts, how='left', on=['week_dt', 'GEOID'])
all_houston = all_houston.fillna(0)

In [27]:
all_houston = all_houston[all_houston['week_dt'] > '12-31-2017']

In [28]:
print(len(all_houston['week_dt'].unique()))
print(all_houston['week_dt'].min())
print(all_houston['week_dt'].max())


print(len(all_dallas['week_dt'].unique()))
print(all_dallas['week_dt'].min())
print(all_dallas['week_dt'].max())

193
2018-01-07 00:00:00
2021-09-12 00:00:00
193
2018-01-07 00:00:00
2021-09-12 00:00:00


In [29]:
conflicts = []

for t in all_houston['week_dt'].unique():
    if( t not in all_dallas['week_dt'].unique()):
        conflicts.append(t)

print(conflicts)

[]


# Now all the historical data from Eviction Lab

In [1445]:
hist_tx_df.head()

,xfileyear,xfileweek,GEOID.Tract,validation,total_filings,total_evictions
0,1993,40,4.843912e+10,NaN,1,1
1,1993,41,4.843910e+10,NaN,1,1
2,1993,42,4.843911e+10,NaN,1,1
3,1993,42,4.843912e+10,NaN,1,1
4,1993,43,4.843910e+10,NaN,1,1


In [1446]:
hist_tx_df = hist_tx_df[hist_tx_df['xfileyear'] > 2000]
hist_tx_df = hist_tx_df[hist_tx_df['GEOID.Tract'].notna()]
hist_tx_df['GEOID.Tract'] = hist_tx_df['GEOID.Tract'].astype(int)
hist_tx_df = hist_tx_df[hist_tx_df['GEOID.Tract'] > 48000000000]
#hist_tx_df['tmp'] = hist_tx_df['xfileyear'].astype(str) + " " + hist_tx_df['xfileweek'].astype(str) + " 1"

# for row in hist_tx_df.itertuples():
#     dt = hist_tx_df.at[row.Index,'tmp']
#     hist_tx_df.at[row.Index, 'week_dt'] =  pd.to_datetime(datetime.datetime.strptime(dt, '%G %V %u'))


In [1447]:
hist_tx_df_week_dt = hist_tx_df.merge(all_weeks_generated, left_on=['xfileweek', 'xfileyear'], right_on=['week', 'year'])

In [1449]:
hist_tx_df_week_dt.head()

,xfileyear,xfileweek,GEOID.Tract,validation,total_filings,total_evictions,week_dt,week,year
0,2001,1,48039661900,NaN,1,1,2001-01-07,1,2001
1,2001,1,48039664300,NaN,1,0,2001-01-07,1,2001
2,2001,1,48039664400,NaN,1,0,2001-01-07,1,2001
3,2001,1,48071710200,NaN,1,1,2001-01-07,1,2001
4,2001,1,48085031658,NaN,1,1,2001-01-07,1,2001


In [1450]:
geoid_list = new_houston['GEOID'].astype(int).unique().tolist()
geoid_list.extend(recent_dal_df['tract_id'].astype(int).unique().tolist())
#geoid_list.extend(hist_tx_df['GEOID.Tract'].astype(int).unique().tolist())

all_tracts_ids = list(set(geoid_list)) 

In [1451]:
len(all_tracts_ids)

1387

In [1452]:

historical_weeks_01_17 = all_weeks_generated[(all_weeks_generated.year > 2000) & (all_weeks_generated.year < 2018)]

all_tracts = []

for week in historical_weeks_01_17['week_dt'].unique():
    #all_tracts.extend(hist_tx_df['GEOID.Tract'].unique())
    all_tracts.extend(all_tracts_ids)

all_tracts.sort()

all_weeks = []
#for tract in hist_tx_df['GEOID.Tract'].unique():
    #all_weeks.extend(hist_tx_df['week_dt'].unique())

for tract in all_tracts_ids:
    all_weeks.extend(historical_weeks_01_17['week_dt'].unique())
    
all_historical = pd.DataFrame({'week_dt':all_weeks, 'GEOID.Tract': all_tracts})

In [1453]:
all_historical.head()

,week_dt,GEOID.Tract
0,2001-01-07,48113000100
1,2001-01-14,48113000100
2,2001-01-21,48113000100
3,2001-01-28,48113000100
4,2001-02-04,48113000100


In [1454]:
all_historical = all_historical.merge(hist_tx_df_week_dt, how='left', on=['week_dt', 'GEOID.Tract'])
all_historical.head()

,week_dt,GEOID.Tract,xfileyear,xfileweek,validation,total_filings,total_evictions,week,year
0,2001-01-07,48113000100,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
1,2001-01-14,48113000100,NaN,NaN,NaN,NaN,NaN,<NA>,NaN
2,2001-01-21,48113000100,2001.0,3.0,NaN,2.0,0.0,3,2001.0
3,2001-01-28,48113000100,2001.0,4.0,NaN,1.0,0.0,4,2001.0
4,2001-02-04,48113000100,NaN,NaN,NaN,NaN,NaN,<NA>,NaN


In [1455]:
all_historical = all_historical.fillna(0)

In [1456]:
all_historical['total_filings'].max()

71.0

In [1457]:
all_historical.head()

,week_dt,GEOID.Tract,xfileyear,xfileweek,validation,total_filings,total_evictions,week,year
0,2001-01-07,48113000100,0.0,0.0,0,0.0,0.0,0,0.0
1,2001-01-14,48113000100,0.0,0.0,0,0.0,0.0,0,0.0
2,2001-01-21,48113000100,2001.0,3.0,0,2.0,0.0,3,2001.0
3,2001-01-28,48113000100,2001.0,4.0,0,1.0,0.0,4,2001.0
4,2001-02-04,48113000100,0.0,0.0,0,0.0,0.0,0,0.0


In [1458]:
tx_hist = pd.DataFrame({'week_dt':all_historical.week_dt, 'GEOID':all_historical['GEOID.Tract'].astype(int), 'count':all_historical.total_filings})

In [1459]:
valid_geoids = np.concatenate([all_dallas['GEOID'].unique(), all_houston['GEOID'].unique()])
print(len(valid_geoids))
tx_hist = tx_hist[tx_hist['GEOID'].isin(valid_geoids)]
print(len(tx_hist['GEOID'].unique()))

1387
1387


In [1460]:
evictions_merged = pd.concat([tx_hist, all_dallas, all_houston])

In [1391]:
evictions_merged.to_csv("houston_dallas_by_tract_2001_2021.csv")

In [1461]:
files = ["demographics/acs_2010.csv", "demographics/acs_2011.csv", "demographics/acs_2012.csv", "demographics/acs_2013.csv", "demographics/acs_2014.csv", "demographics/acs_2015.csv", "demographics/acs_2016.csv", "demographics/acs_2017.csv", "demographics/acs_2018.csv", "demographics/acs_2019.csv"]
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

all_acs = []

def amalgamate_data( file, year ):
    acs = pd.read_csv(file)
    acs = acs[acs['Geo_FIPS'].isin(all_geoids)]
    acs = acs.assign(year=year)
    all_acs.append(acs)
    
for f, y in zip(files, years):
    amalgamate_data(f, y)

In [1462]:
from functools import reduce

acs_merged = reduce(lambda left,right: pd.concat([left,right], axis=0), all_acs)

In [1394]:
cols = ['Geo_FIPS', 'year', 'SE_A03001_003', 'SE_A18005_001', 'SE_A06001_005', 'SE_B18002_002', 'SE_B18002_003', 'SE_A08002B_003', 'SE_A10011_003', 'SE_A10039B_009']

#for this experiment, don't merge
#acs_merged = acs_merged.filter(cols)

In [1463]:
acs_merged = acs_merged.fillna(0)
acs_merged = acs_merged.replace(np.nan, 0)
acs_merged.rename(columns={'Geo_FIPS':'GEOID'}, inplace=True)

In [1464]:
#merged['year'] = pd.to_datetime(merged['week_dt'])
evictions_merged['year'] = evictions_merged['week_dt'].dt.year
evictions_merged['week'] = evictions_merged['week_dt'].dt.isocalendar().week

evictions_merged.loc[(evictions_merged['week_dt'].dt.month == 1) & (evictions_merged['week'] == 52), 'week'] = 0
evictions_merged.loc[(evictions_merged['week_dt'].dt.month == 1) & (evictions_merged['week'] == 53), 'week'] = 0

In [1465]:
evictions_merged.head()

,week_dt,GEOID,count,year,week
0,2001-01-07,48113000100,0.0,2001,1
1,2001-01-14,48113000100,0.0,2001,2
2,2001-01-21,48113000100,2.0,2001,3
3,2001-01-28,48113000100,1.0,2001,4
4,2001-02-04,48113000100,0.0,2001,5


In [1466]:
weeks_10_19 = all_weeks_generated[(all_weeks_generated.year > 2009) & (all_weeks_generated.year < 2020)].copy()
weeks_10_19.loc[(weeks_10_19['week_dt'].dt.month == 1) & (weeks_10_19['week'] == 52), 'week'] = 0
weeks_10_19.loc[(weeks_10_19['week_dt'].dt.month == 1) & (weeks_10_19['week'] == 53), 'week'] = 0


In [1467]:
weeks_10_19

,week_dt,week,year
469,2010-01-03,0,2010
470,2010-01-10,1,2010
471,2010-01-17,2,2010
472,2010-01-24,3,2010
473,2010-01-31,4,2010
...,...,...,...
986,2019-12-01,48,2019
987,2019-12-08,49,2019
988,2019-12-15,50,2019
989,2019-12-22,51,2019


In [1468]:
evictions_merged = evictions_merged.drop(labels=['week_dt'], axis='columns')

In [1469]:
evictions_merged_test = evictions_merged.merge(weeks_10_19, on=['week', 'year'])

In [1470]:
#evictions_merged_test[(evictions_merged_test.year == 2017) & (evictions_merged_test.week == 52)].head(100)
evictions_merged_test[evictions_merged_test.duplicated(subset=['week', 'year', 'GEOID'], keep=False)]
#evictions_merged_test.duplicated()

,GEOID,count,year,week,week_dt


In [1471]:
acs_merged.head()

,GEOID,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,SE_A11003_003,SE_A11003_004,SE_A11003_005,SE_A11003_006,SE_A11003_007,SE_A11003_008,SE_A11003_009,SE_A11003_010,SE_A11003_011,SE_A11003_012
0,48113000100,14000US48113000100,"Census Tract 1, Dallas County, Texas","Census Tract 1, Dallas County, Texas",tx,140,0,ACSSF,15500,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,48113000201,14000US48113000201,"Census Tract 2.01, Dallas County, Texas","Census Tract 2.01, Dallas County, Texas",tx,140,0,ACSSF,15501,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,48113000202,14000US48113000202,"Census Tract 2.02, Dallas County, Texas","Census Tract 2.02, Dallas County, Texas",tx,140,0,ACSSF,15502,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48113000300,14000US48113000300,"Census Tract 3, Dallas County, Texas","Census Tract 3, Dallas County, Texas",tx,140,0,ACSSF,15503,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,48113000401,14000US48113000401,"Census Tract 4.01, Dallas County, Texas","Census Tract 4.01, Dallas County, Texas",tx,140,0,ACSSF,15504,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1472]:
# merged = merged[merged['year'] > 2008]
# merged = merged[merged['year'] < 2020]
evictions_merged_test['year'] = evictions_merged_test['year'].astype(int)
evictions_merged_test['GEOID'] = evictions_merged_test['GEOID'].astype(int)
acs_merged['year'] = acs_merged['year'].astype(int)
acs_merged['GEOID'] = acs_merged['GEOID'].astype(int)
evictions_acs = evictions_merged_test.merge(acs_merged, how='left', on=['GEOID', 'year'])

In [1473]:
evictions_acs.head()

,GEOID,count,year,week,week_dt,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,...,SE_A11003_003,SE_A11003_004,SE_A11003_005,SE_A11003_006,SE_A11003_007,SE_A11003_008,SE_A11003_009,SE_A11003_010,SE_A11003_011,SE_A11003_012
0,48113000100,0.0,2010,0,2010-01-03,14000US48113000100,"Census Tract 1, Dallas County, Texas","Census Tract 1, Dallas County, Texas",tx,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,48113000201,0.0,2010,0,2010-01-03,14000US48113000201,"Census Tract 2.01, Dallas County, Texas","Census Tract 2.01, Dallas County, Texas",tx,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,48113000202,0.0,2010,0,2010-01-03,14000US48113000202,"Census Tract 2.02, Dallas County, Texas","Census Tract 2.02, Dallas County, Texas",tx,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48113000300,0.0,2010,0,2010-01-03,14000US48113000300,"Census Tract 3, Dallas County, Texas","Census Tract 3, Dallas County, Texas",tx,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,48113000401,0.0,2010,0,2010-01-03,14000US48113000401,"Census Tract 4.01, Dallas County, Texas","Census Tract 4.01, Dallas County, Texas",tx,140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1474]:
print(len(evictions_acs))
len(evictions_acs) - len(evictions_acs[evictions_acs['SE_A03001_003'].isna()])

724014


460404

In [1475]:
print(len(evictions_acs['GEOID'].unique()) - len(evictions_acs[evictions_acs['SE_A03001_003'].isna()]['GEOID'].unique()))

882


In [1476]:
len(evictions_acs['GEOID'].unique())

1387

In [1477]:
evictions_acs_clean = evictions_acs.dropna()

In [1479]:
#evictions_acs_clean.to_csv("dallas_houston_evictions_ACS_10_19.csv")
evictions_acs_clean.to_csv("evictions_ACS__ALL_10_19.csv")

Need to get the average cost of the filing for each census tract in Dallas and Houston

In [35]:

mean_claims = []

for geoid in new_houston["GEOID"].unique():
    mean_claims.append( {"GEOID":geoid, "claim":new_houston[new_houston["GEOID"] == geoid]["Claim Amount"].mean()} )
    
for geoid in recent_dal_df["tract_id"].unique():
    mean_claims.append( {"GEOID":geoid, "claim":recent_dal_df[recent_dal_df["tract_id"] == geoid]["amount"].mean()} )
    

In [38]:
mean_claims_df = pd.DataFrame(mean_claims)
mean_claims_df.to_csv("mean_claim_by_geoid.csv")